In [2]:
import sys
sys.path.append("/home/chihong/Oxebfe/MyLib")

In [20]:
import re, json
import Request, Parse

In [34]:
Host = "tamuctf.com"
TLS = True
USER = "quriosity"
PASSWORD = "quriosity"
HEADER = {}

In [74]:
def Login():
    path = "/"
    url = Request.MakeURL(Host, path, {}, TLS)
    (resp_header, resp_body) = Request.Request(url)
    session_cookie = ""
    for item in resp_header:
        if item[0] == "Set-Cookie" and "session" in item[1]:
            session_cookie = item[1].split(";")[0]
    #print(session_cookie)
    csrf_nonce = ""
    match = re.findall("csrf_nonce = \"([0-9a-f]+?)\"", resp_body.decode("utf-8"))
    if len(match) > 0:
        csrf_nonce = match[0]
    #print(csrf_nonce)
    
    path = "/login"
    url = Request.MakeURL(Host, path, {}, TLS)
    header = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Cookie": session_cookie,
    }
    query = {
        "name": USER,
        "password": PASSWORD,
        "nonce": csrf_nonce,
    }
    body = Request.MakeBody(query, header)
    (resp_header, resp_body) = Request.Request(url, body, header)
    for item in resp_header:
        if item[0] == "Set-Cookie" and "session" in item[1]:
            session_cookie = item[1].split(";")[0]
    #print(session_cookie)
    HEADER["cookie"] = session_cookie
    return

def ListChals():
    path = "/chals"
    url = Request.MakeURL(Host, path, {}, TLS)
    (resp_header, resp_body) = Request.Request(url, None, HEADER)
    chals = json.loads(resp_body.decode("utf-8"))
    return chals

def ChalSolves():
    path = "/chals/solves"
    url = Request.MakeURL(Host, path, {}, TLS)
    (resp_header, resp_body) = Request.Request(url, None, HEADER)
    solves = json.loads(resp_body.decode("utf-8"))
    return solves

def TeamSolves(num):
    path = "/solves/" + str(num)
    url = Request.MakeURL(Host, path, {}, TLS)
    (resp_header, resp_body) = Request.Request(url, None, HEADER)
    solves = json.loads(resp_body.decode("utf-8"))
    return solves

def ScoreBoard():
    path = "/scores"
    url = Request.MakeURL(Host, path, {}, TLS)
    (resp_header, resp_body) = Request.Request(url, None, HEADER)
    result = json.loads(resp_body.decode("utf-8"))
    return result
        
def DetailChal(num):
    path = "/chals/" + str(num)
    url = Request.MakeURL(Host, path, {}, TLS)
    (resp_header, resp_body) = Request.Request(url, None, HEADER)
    chal = json.loads(resp_body.decode("utf-8"))
    return chal

In [36]:
Login()
print(HEADER)

11:12:22 Request Start: https://tamuctf.com/
11:12:22 Request End: 200, 4658 Bytes
11:12:23 Request Start: https://tamuctf.com/login
11:12:23 Request End: 302, 229 Bytes
{'cookie': 'session=.eJwlj8GKgzAURX-lvLUbxQ5FmF2sNPBSlLSS7Bx16kt8ltYpHVP67yPM9h4Oh_uCpmOaIPtuxrmPgDrIdh9pBNN1anvIXrD5ggzQ5QnqckE9MtZywEKOKCTZ4rQ1rt1iWHddDVZLQtetfE9YV4PiU4LB_CLvndF2NFwxhktQXJEqJClxCZYPiQnWH3UZkE2snE-wxsXotRHK2DI-Lefxv3v2Vp-90u3T8CE-Clys6DyKPDUuT5XAT3hH8Jj7-9TwegBujztdZ_pZNvD-A8YbUDU.D1OPBw.w62hfUgaaslO6rYkg6bKhh5lmIM'}


In [135]:
chals = ListChals()
chals = chals["game"]

07:08:27 Request Start: https://tamuctf.com/chals
07:08:27 Request End: 200, 32364 Bytes


In [136]:
solves = ChalSolves()
for chal in chals:
    chal["solves"] = solves[str(chal["id"])]

07:08:29 Request Start: https://tamuctf.com/chals/solves
07:08:29 Request End: 200, 577 Bytes


In [137]:
chals = sorted(chals, key=lambda x: x["solves"], reverse=True)
for chal in chals:
    print("{id:>4} {solves:>4} {type:<24} {category:16} {name}".format(**chal))

   1  956 standard                 Misc             Howdy!
  13  748 dynamic                  Misc             Who am I?
  14  706 dynamic                  Misc             Who do I trust?
  25  641 dynamic                  Web              Not Another SQLi Challenge
  18  588 dynamic                  Reversing        Cheesy
  23  579 dynamic                  Misc             Where am I?
  19  470 dynamic                  Reversing        Snakes over cheese
   4  439 dynamic                  Crypto           -.-
  58  391 multiquestionchallenge   MicroServices    0_intrusion
  29  374 dynamic                  Web              Robots Rule
  38  337 multiquestionchallenge   DriveByInc       0_intrusion
  32  326 dynamic                  Pwn              Pwn1
  35  294 dynamic                  Pwn              Pwn4
  44  272 multiquestionchallenge   ReadingRainbow   0_Network_Enumeration
  27  235 dynamic                  Web              Science!
   2  233 dynamic                  Misc  

In [103]:
chalid = 4
result = DetailChal(chalid)
print("{category}: {name}\n{type} {initial} {value}\n{description}\n{files}".format(**result))

12:18:13 Request Start: https://tamuctf.com/chals/4
12:18:13 Request End: 200, 981 Bytes
Crypto: -.-
dynamic 500 436
To 1337-H4X0R:

Our coworker Bob loves a good classical cipher. Unfortunately, he also loves to send everything encrypted with these ciphers. Can you go ahead and decrypt this for me?

Difficulty: easy
['cd5fef538fad2500c30c4ea389c50244/flag.txt']


In [104]:
chalid = 7
result = DetailChal(chalid)
print("{category}: {name}\n{type} {initial} {value}\n{description}\n{files}".format(**result))

12:18:57 Request Start: https://tamuctf.com/chals/7
12:18:57 Request End: 200, 848 Bytes
Crypto: :)
dynamic 500 497
Look at what I found! `XUBdTFdScw5XCVRGTglJXEpMSFpOQE5AVVxJBRpLT10aYBpIVwlbCVZATl1WTBpaTkBOQFVcSQdH`

Difficulty: easy
[]


In [105]:
chalid = 22
result = DetailChal(chalid)
print("{category}: {name}\n{type} {initial} {value}\n{description}\n{files}".format(**result))

12:20:29 Request Start: https://tamuctf.com/chals/22
12:20:29 Request End: 200, 1117 Bytes
Crypto: RSAaaay
dynamic 500 487
Hey, you're a hacker, right? I think I am too, look at what I made!

-----------


`(2531257, 43)`

My super secret message: `906851 991083 1780304 2380434 438490 356019 921472 822283 817856 556932 2102538 2501908 2211404 991083 1562919 38268`

-----------

Problem is, I don't remember how to decrypt it... could you help me out?

Difficulty: easy
[]


In [106]:
chalid = 6
result = DetailChal(chalid)
print("{category}: {name}\n{type} {initial} {value}\n{description}\n{files}".format(**result))

12:20:52 Request Start: https://tamuctf.com/chals/6
12:20:52 Request End: 200, 1043 Bytes
Crypto: Holey Knapsack
dynamic 500 500
My knapsack has a hole in it

Cipher text: `11b90d6311b90ff90ce610c4123b10c40ce60dfa123610610ce60d450d000ce61061106110c4098515340d4512361534098509270e5d09850e58123610c9`

Public key: `{99, 1235, 865, 990, 5, 1443, 895, 1477}`

The flag is slightly off format.

Difficulty: medium

[]


In [107]:
chalid = 5
result = DetailChal(chalid)
print("{category}: {name}\n{type} {initial} {value}\n{description}\n{files}".format(**result))

12:21:02 Request Start: https://tamuctf.com/chals/5
12:21:02 Request End: 200, 1416 Bytes
Crypto: Mike's Marvelous Mystery Curves
dynamic 500 500
Mike, the System Administrator, thought it would be a good idea to implement his own Elliptic Curve Diffie Hellman key exchange using unnamed curves to use across the network. We managed to capture network traffic of the key exchange along with an encrypted file transfer. See if you can read the contents of that file.

Note: The password to the AES192-CBC encrypted file is the shared key x and y coordinates from the key exchange concatenated together. (e.g. sharedKey = (12345,67890)  password = "1234567890")

Difficulty: hard

Edit: 02/23/2019 14:33
Changed AES256-CBC to AES192-CBC
['956b64d4af0dbd3db928c61baaac0df1/key_exchange.pcap']


In [138]:
result = ScoreBoard()
standings = result["standings"]
for item in standings:
    print("{pos:>8} {id:>4} {team:32} {score:>8}".format(**item))

07:09:35 Request Start: https://tamuctf.com/scores
07:09:35 Request End: 200, 57317 Bytes
       1  495 dcua                                24002
       2  569 NYUSEC                              21502
       3  394 FireShell                           21502
       4  584 Armia Prezesa                       21102
       5  525 WreckTheLine                        21002
       6  484 OpenToAll                           20702
       7  559 perfect blue                        20702
       8  790 justCatTheFish                      20502
       9 1128 Aperi'Kube                          19802
      10  565 Made In MIM                         19406
      11  657 ELT                                 18508
      12  445 voidka                              18407
      13  654 Sudo_root                           18402
      14  718 Tr0j4nh0r53                         18310
      15   66 BlackPanthers                       18002
      16  496 noraneco                            17816
      17  402 

In [139]:
num = 99
result = TeamSolves(num)
solves = result["solves"]
for item in solves:
    print("{chalid:>4} {category:16} {value:>4} {chal:32} {time:>12}".format(**item))

07:10:01 Request Start: https://tamuctf.com/solves/99
07:10:02 Request End: 200, 675 Bytes
   1 Misc                1 Howdy!                             1550880455
  18 Reversing         255 Cheesy                             1550906724
  19 Reversing         344 Snakes over cheese                 1550906740
  51 Reversing         487 NoCCBytes                          1550906788
  22 Crypto            470 RSAaaay                            1550906807
  50 Reversing         485 KeyGenMe                           1550934263
  14 Misc              146 Who do I trust?                    1550934420


In [140]:
num = 864
result = TeamSolves(num)
solves = result["solves"]
for item in solves:
    print("{chalid:>4} {category:16} {value:>4} {chal:32} {time:>12}".format(**item))

07:10:03 Request Start: https://tamuctf.com/solves/864
07:10:04 Request End: 200, 960 Bytes
   1 Misc                1 Howdy!                             1550976631
  22 Crypto            470 RSAaaay                            1550979915
  13 Misc              103 Who am I?                          1550985695
  23 Misc              263 Where am I?                        1550985835
  14 Misc              146 Who do I trust?                    1550985915
  25 Web               208 Not Another SQLi Challenge         1550993995
  18 Reversing         255 Cheesy                             1550994483
  58 MicroServices     100 0_intrusion                        1550996873
   3 Misc              470 Hello World                        1551042454
   4 Crypto            364 -.-                                1551045762
